In [1]:
import tensorflow as tf
import os
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

#import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools
import cv2
import itertools as it
from glob import glob
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import tensorflow_hub as hub

#####################################################

def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

#####################################################


def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

#####################################################


def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)
    
#####################################################

hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

vid = cv2.VideoCapture(0)

while(True):
    
    ret, frame = vid.read()
    cv2.imwrite("new_content.jpg", frame)
    content_image = load_img('new_content.jpg')
    
    #new_target_image = tf.Variable(new_content)
    #combined_target = tf.Variable(tf.math.abs(tf.math.subtract(new_target_image, target_image))*0.9)
    style_image = load_img('C:/Users/ralna/OneDrive/Desktop/artathon/Neural-Style-Transfer-master/images/inputs/style/misty-mood-leonid-afremov.jpg')
    
    stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
    
    #im=tensor_to_image(stylized_image)
    
    im = np.squeeze(stylized_image, 0 )
    imS = cv2.resize(im, (1000, 1000)) 
    cv2.imshow('frame', imS)
    
    #target_image = combined_target
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
    # After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

